In [1]:
from pyspark import SparkContext, SparkConf
sc = SparkContext(appName="YourTest", master="local[2]")

In [7]:
from simple_tokenize import simple_tokenize
from math import log10

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    def pairs(x):
        my_list = set(simple_tokenize(x))
        pairs = []
        for x in my_list:
                for y in my_list:
                    #skip the pair with the same word
                    if x != y:
                        pairs.append((x, y))
        return pairs
    lines = sc.textFile("Shakespeare.txt")
    numberlines = lines.count()
    tokens = lines.flatMap(lambda x:set(simple_tokenize(x))).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
    p = lines.flatMap(lambda x:pairs(x)).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
    p = p.filter(lambda x:x[1]>threshold)
    p = p.map(lambda x: (x[0][0], x)).join(tokens).map(lambda x: (x[1][0][0][1], x[1])).join(tokens)
    p = p.map(lambda x: (x[1][0][0][0], x[1][0][0][1], x[1][0][1], x[1][1]))
    pmi = p.map(lambda x:(x[0], log10((x[1]/numberlines)/((x[2]/numberlines)*(x[3]/numberlines))),x[1],x[2],x[3]))
    pmi = pmi.sortBy(lambda x:x[1], False)
    return pmi.collect()

In [8]:
from simple_tokenize import simple_tokenize
from math import log10

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    def pairs(x):
        my_list = set(simple_tokenize(x))
        pairs = []
        for x in my_list:
                for y in my_list:
                    #skip the pair with the same word
                    if x != y:
                        pairs.append((x, y))
        return pairs
    lines = sc.textFile("Shakespeare.txt")
    numberlines = lines.count()
    tokens = lines.flatMap(lambda x:set(simple_tokenize(x))).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
    p = lines.flatMap(lambda x:pairs(x)).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
    p = p.filter(lambda x:x[1]>threshold)
    p = p.map(lambda x: (x[0][0], x)).join(tokens).map(lambda x: (x[1][0][0][1], x[1])).join(tokens)
    p = p.map(lambda x: (x[1][0][0][0], x[1][0][0][1], x[1][0][1], x[1][1]))
    pmi = p.map(lambda x:(x[0][0],(x[0], log10((x[1]/numberlines)/((x[2]/numberlines)*(x[3]/numberlines))),x[1],x[2],x[3])))
    pmi = pmi.groupByKey().cache()
    return pmi.map(lambda x:(x[0],list(x[1]))).takeSample(False,samp_size)
